### Máquina de Soporte Vectorial (SVM)

Importar librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Cargar dataset

In [2]:
df = pd.read_csv('../data/airline_cleaned.csv')
df.head(2)

,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,...,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Gender,Customer Type,Type of Travel,Class,satisfaction
0,13.0,460.0,3.0,4.0,3.0,1.0,5.0,3.0,5.0,5.0,...,4.0,5.0,5.0,25.0,18.0,1,0,1,2,0
1,25.0,235.0,3.0,2.0,3.0,3.0,1.0,3.0,1.0,1.0,...,1.0,4.0,1.0,1.0,6.0,1,1,0,0,0


Separar las Características (X) y la Etiqueta (y)

+ y = satisfaction (variable a predecir)
* X 

In [3]:
X = df.drop(columns=['satisfaction'])
y = df['satisfaction']

#### Dividir el Conjunto de Datos

Se divide el dataset en conjuntos de entrenamiento y prueba:

* X_train, y_train: Se utilizan para entrenar el modelo (80% de los datos).

* X_test, y_test: Se utilizan para evaluar el modelo (20% de los datos).

* test_size=0.20: Indica que el 20% de los datos se reservarán para pruebas.

* random_state=0: Para asegurar que la división sea reproducible.

#### Entrenamiento del Modelo


In [4]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lista de valores de C para probar
C_values = [0.1, 1, 10]
kernels = ['linear', 'rbf', 'poly']

# Diccionarios para almacenar los resultados
results = {kernel: {'train_scores': [], 'cv_scores': [], 'test_scores': []} for kernel in kernels}

for kernel in kernels:
    for C in C_values:
        # Crear y entrenar el modelo
        svm = SVC(C=C, kernel=kernel, random_state=42)
        
        # Calcular puntuación de entrenamiento
        svm.fit(X_train, y_train)
        train_score = accuracy_score(y_train, svm.predict(X_train))
        results[kernel]['train_scores'].append(train_score)
        
        # Calcular puntuación de validación cruzada
        cv_score = np.mean(cross_val_score(svm, X_train, y_train, cv=5))
        results[kernel]['cv_scores'].append(cv_score)
        
        # Calcular puntuación de prueba
        test_score = accuracy_score(y_test, svm.predict(X_test))
        results[kernel]['test_scores'].append(test_score)

# Encontrar la mejor combinación de kernel y C
best_kernel = ''
best_C = 0
best_score = 0

for kernel in kernels:
    max_score_index = np.argmax(results[kernel]['cv_scores'])
    if results[kernel]['cv_scores'][max_score_index] > best_score:
        best_score = results[kernel]['cv_scores'][max_score_index]
        best_kernel = kernel
        best_C = C_values[max_score_index]

# Graficar los resultados
plt.figure(figsize=(15, 5))
for i, kernel in enumerate(kernels):
    plt.subplot(1, 3, i+1)
    plt.plot(C_values, results[kernel]['train_scores'], label='Train')
    plt.plot(C_values, results[kernel]['cv_scores'], label='Cross-validation')
    plt.plot(C_values, results[kernel]['test_scores'], label='Test')
    plt.xscale('log')
    plt.xlabel('C')
    plt.ylabel('Accuracy')
    plt.title(f'SVM con kernel {kernel}')
    plt.legend()
plt.tight_layout()
plt.show()

print(f"El mejor kernel es: {best_kernel}")
print(f"El mejor valor de C es: {best_C}")
print(f"Precisión de validación cruzada con los mejores parámetros: {best_score:.4f}")

# Entrenar el modelo final con los mejores parámetros
best_svm = SVC(C=best_C, kernel=best_kernel, random_state=42)
best_svm.fit(X_train, y_train)

# Evaluar en el conjunto de prueba
final_accuracy = accuracy_score(y_test, best_svm.predict(X_test))
print(f"Precisión final en el conjunto de prueba: {final_accuracy:.4f}")

# Calcular el overfitting
train_error = 1 - accuracy_score(y_train, best_svm.predict(X_train))
test_error = 1 - final_accuracy
overfitting = test_error - train_error
print(f"Overfitting: {overfitting:.4f}")

#### Evaluación del Modelo (Predicciones)

Se hacen predicciones sobre el conjunto de prueba usando el modelo entrenado.

* Informe de Clasificación y Matriz de Confusión

    * classification_report: Muestra varias métricas de evaluación, como precisión (accuracy), recall, F1-score, etc.

    * confusion_matrix: Muestra la matriz de confusión, que ayuda a entender cómo de bien el modelo está clasificando las etiquetas correctas frente a las incorrectas.

In [ ]:
# Evaluar el modelo
from sklearn.metrics import classification_report, confusion_matrix

y_pred = best_svm.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


        CONCLUSION: 

        En general, los resultados de clasificación son bastante buenos. La precisión, recall y F1-score son altos para ambas categorías, lo que indica que el modelo está clasificando correctamente la mayoría de los casos. La precisión general también es alta, lo que significa que el modelo está realizando una buena clasificación en general.


        Resumen de los resultados:

        * Precisión: Mide la proporción de casos correctamente clasificados entre todos los casos clasificados como esa categoría. En este caso, la precisión para "neutral o insatisfecho" es de 0,92, lo que significa que el 92% de los casos clasificados como "neutral o insatisfecho" lo eran realmente. La precisión para "satisfecho" también es de 0,92.

        * Recall: Mide la proporción de casos correctamente clasificados de una categoría entre todos los casos que realmente pertenecen a esa categoría. Aquí, el recall para "neutral o insatisfecho" es de 0,94, indicando que el 94% de los casos que realmente eran "neutral o insatisfecho" fueron correctamente clasificados. El recall para "satisfecho" es de 0,89.

        * F1-score: Es la media armónica de precisión y recall, proporciona un equilibrio entre ambos. En este caso, el F1-score para ambas categorías es de 0,92.

        * Accuracy: Mide la proporción total de casos correctamente clasificados entre todos los casos. La precisión general es de 0,92, lo que significa que el 92% de todos los casos fueron clasificados correctamente.
        Matriz de confusión: Muestra la distribución de casos clasificados correctamente e incorrectamente entre las categorías. Por ejemplo, la entrada [13900, 834] indica que 13900 casos fueron correctamente clasificados como "neutral o insatisfecho" y 834 casos fueron incorrectamente clasificados como "neutral o insatisfecho" cuando en realidad eran "satisfechos".



#### Curva ROC y AUC 

In [ ]:
from sklearn.metrics import roc_curve, auc

def calculate_roc(best_knnsvm, X_test, y_test):
    
    # Obtener las probabilidades predichas para la clase positiva
    y_pred_proba = best_svm.predict_proba(X_test)[:, 1]
    
    # Calcular la curva ROC
    fpr, tpr, thresholds  = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    return fpr, tpr, roc_auc

# Paso 3: Graficar la curva ROC
def plot_roc(fpr, tpr, roc_auc):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()
    

# Uso de las funciones
fpr, tpr, roc_auc = calculate_roc(best_svm, X_test, y_test)
plot_roc(fpr, tpr, roc_auc)

# Opcional: Imprimir los valores de y_test para verificar
print("Valores únicos en y_test:", y_test.unique())
print("Conteo de valores en y_test:\n", y_test.value_counts())

        CONCLUSIÓN:
        
        La curva ROC y el valor AUC indican que el modelo de clasificación utilizado tiene un excelente desempeño.Es capaz de distinguir con alta precisión entre las dos clases.

#### Validación Cruzada

In [ ]:
from sklearn.model_selection import cross_val_score


scores = cross_val_score(best_svm, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-Validation Accuracy Scores:", scores)
print("Mean Accuracy:", scores.mean())


    CONCLUSION:

    Según los resultados obtenidos, el rendimiento del modelo en la validación cruzada es relativamente constante en los distintos pliegues. La precisión media de aproximadamente 0,9183 indica que el modelo está logrando un alto nivel de precisión en promedio.